# setup

In [12]:
%matplotlib inline
from eelbrain import *
import scipy, mne, os, shutil, pdb, importlib
import numpy as np
import matplotlib.pyplot as plt

root_folder = 'data_path' # path to DRUM dataset
subjects_dir = f'{root_folder}/mri' # copy freesurfer fsaverage files here
meg_folder = f'{root_folder}/meg'
output_folder = 'output_path' # path to output folder
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
subjects = [f for f in os.listdir(meg_folder) if f[0]=='R'] # get subject list
subjects.sort()

# make sourcespace

In [ ]:
# make subjects_dir mri folders and scaled source spaces
# freesurfer fsaverage files need to be in the subjects_dir
for subject in subjects:
    if not os.path.exists(f'{subjects_dir}/{subject}/bem/{subject}-ico-4-src.fif'):
        print(f'making {subject}')
        os.makedirs(f'{subjects_dir}/{subject}/bem')
        shutil.copyfile(f'{meg_folder}/{subject}/MRI scaling parameters.cfg', f'{subjects_dir}/{subject}/MRI scaling parameters.cfg')
        mne.scale_source_space(subject, f'{{subject}}-ico-4-src.fif', subjects_dir=subjects_dir)

# make resting beta power

In [ ]:
for subject in subjects:
    if subject == 'R26672': # fix for R2667 is not needed for resting data
        continue
    for visit in ['visit1', 'visit2']:
        fwdfile = f'{meg_folder}/{subject}/{subject}_{visit}_resting-ico-4-fwd.fif'
        snds = []
        for i in range(1,3): # loop over resting 1 and 2
            rawfile = f'{meg_folder}/{subject}/{subject}_{visit}_resting{i}-raw.fif'
            if not os.path.exists(rawfile):
                print(f'file not found: {subject}_{visit}_resting{i}')
                continue
            print(f'loading {subject}_{visit}_resting{i}')
            raw = mne.io.read_raw_fif(rawfile)
            
            # source localization
            covfile = f'{rawfile[:-7]}cov.fif'
            fwd = mne.read_forward_solution(fwdfile)
            cov = mne.read_cov(covfile)
            invsol = mne.minimum_norm.make_inverse_operator(raw.info, fwd, cov, fixed=True, depth=0.8)
            stc1 = mne.minimum_norm.apply_inverse_raw(raw, invsol, lambda2 = 1, method='MNE')
            snd1 = load.fiff.stc_ndvar(stc1, 'fsaverage', 'ico-4', subjects_dir=subjects_dir)
            snds.append(snd1.sub(time=(5, snd1.time.tmax-5)))
            del stc1, snd1
        if len(snds) == 0:
            continue
        # concatenate resting 1 and 2
        snd1 = concatenate(snds, 'time')
        del snds
        
        # make psd on 15s blocks of the data
        nblocks = snd1.time.tmax/15
        psds = []
        for i in range(int(np.floor(nblocks))):
            print('making psd block', i*15, 's - ', (i+1)*15, 's', ' '*20, end='\r')
            psds.append(psd_welch(snd1.sub(time=(i*15,(i+1)*15)), n_per_seg=256, n_overlap=128).sub(frequency=(1,40)))
        psd = combine(psds).mean('case')
        
        save.pickle(psd, f'{output_folder}/{subject}_{visit}_resting_psd.pkl')
        
        # plot average psds across the whole brain and across rolandic roi
        rolandicROI = list(set([l for l in psd.source.parc.as_labels() if 'central' in l]))
        psd1 = psd.mean('source') # whole brain average
        psd1.name = 'whole brain'
        psd2 = psd.sub(source=rolandicROI).mean('source') # rolandic roi average
        psd2.name = 'rolandic ROI'
        p = plot.UTS([[psd1, psd2]])
        p.save(f'{output_folder}/plots_psd_{subject}_{visit}_resting.png')
        p.close()

# make visual beta power

In [ ]:
tasks = ['fam', 'mm', 'pn', 'pd']

for subject in subjects:
    for visit in ['visit1', 'visit2']:
        fwdfile = f'{meg_folder}/{subject}/{subject}_{visit}_visual-fwd.fif'
        for task in tasks:
            rawfile = f'{meg_folder}/{subject}/{subject}_{visit}_visual_{task}-raw.fif'
            if not os.path.exists(rawfile):
                print(f'file not found: {subject}_{visit}_visual_{task}')
                continue
            print(f'loading {subject}_{visit}_visual_{task}')
            raw = mne.io.read_raw_fif(rawfile)
            
            # source localization
            covfile = f'{rawfile[:-7]}cov.fif'
            fwd = mne.read_forward_solution(fwdfile)
            cov = mne.read_cov(covfile)
            invsol = mne.minimum_norm.make_inverse_operator(raw.info, fwd, cov, fixed=True, depth=0.8)
            stc1 = mne.minimum_norm.apply_inverse_raw(raw, invsol, lambda2 = 1, method='MNE')
            snd1 = load.fiff.stc_ndvar(stc1, 'fsaverage', 'ico-4', subjects_dir=subjects_dir)

            # make psd in 15s blocks
            nblocks = snd1.time.tmax/15
            psds = []
            for i in range(int(np.floor(nblocks))):
                print('making psd block', i*15, 's - ', (i+1)*15, 's', ' '*20, end='\r')
                psds.append(psd_welch(snd1.sub(time=(i*15,(i+1)*15)), n_per_seg=256, n_overlap=128).sub(frequency=(1,40)))
            psd = combine(psds).mean('case')
            save.pickle(psd, f'{output_folder}/{subject}_{visit}_visual_{task}_psd.pkl')

            # plot average psds across the whole brain and across rolandic roi
            rolandicROI = list(set([l for l in psd.source.parc.as_labels() if 'central' in l]))
            psd1 = psd.mean('source') # whole brain average
            psd1.name = 'whole brain'
            psd2 = psd.sub(source=rolandicROI).mean('source') # rolandic roi average
            psd2.name = 'rolandic ROI'
            p = plot.UTS([[psd1, psd2]])
            p.save(f'{output_folder}/plots_psd_{subject}_{visit}_visual_{task}.png')
            p.close()

# write CSV file beta power

In [ ]:
CONTROLS = ['R2517', 'R2519', 'R2520', 'R2521', 'R2525', 'R2528', 'R2496', 'R2673',] 
PATIENTS = ['R2527', 'R2540', 'R2546', 'R2598', 'R2615', 'R2617', 'R2664', 'R2667', 'R2668',]

# lesion hemisphere
LEFT =  ['R2527', 'R2540', 'R2667', 'R2668']
RIGHT = ['R2546', 'R2598', 'R2615', 'R2617', 'R2664']

psdsubj = dict(C={},P={})
beta_band = (13, 25)
psdrange = (2, 40)


outfile = 'betapower.csv'
with open(outfile, 'w+') as f:
    f.write(f'subject,group,rel_beta,visit,task\n') # column headings
    
# lesion hemisphere
outfile2 = 'betapower_lesionhemi.csv'
with open(outfile2, 'w+') as f:
    f.write(f'subject,group,rel_beta,hemi,lesion,visit,task\n') # column headings 
    
for subject in subjects:
    for visit in ['visit1', 'visit2']:
        for task in ['resting', 'visual_fam', 'visual_mm', 'visual_pn', 'visual_pd']:
            infile = f'{subject}_{visit}_{task}_psd.pkl'
            if not os.path.exists(f'{output_folder}/{infile}'):
                print(f'file not found: {infile}')
                continue
            print(f'loading: {infile}')
            psd = load.unpickle(f'{output_folder}/{infile}')
            
            subject = subject[:5] # this is to combine R26672 and R2667
            if subject in CONTROLS:
                group = 'C'
            else:
                group = 'P'
            
            # rolandic ROI
            rolandicROI = list(set([l for l in psd.source.parc.as_labels() if 'central' in l]))
            psd2 = psd.sub(source=rolandicROI).mean('source').sub(frequency=psdrange)

            # relative power
            psd_rel = psd2.copy()
            psd_rel.x /= np.sum(psd_rel.x)
            rel_beta = psd_rel.sub(frequency=beta_band).sum('frequency')

            with open(outfile, 'a+') as f:
                f.write(f'{subject},{group},{rel_beta},{visit},{task}\n')
                    
            psdL = psd.sub(source=rolandicROI).sub(source='lh').mean('source').sub(frequency=psdrange)
            psd_relL = psdL.copy()
            psd_relL.x /= np.sum(psd_relL.x)
            rel_betaL = psd_relL.sub(frequency=beta_band).sum('frequency')

            psdR = psd.sub(source=rolandicROI).sub(source='rh').mean('source').sub(frequency=psdrange)
            psd_relR = psdR.copy()
            psd_relR.x /= np.sum(psd_relR.x)
            rel_betaR = psd_relR.sub(frequency=beta_band).sum('frequency')

            with open(outfile2, 'a+') as f:
                if subject in LEFT:
                    f.write(f'{subject},{group},{rel_betaL},left,ipsi,{visit},{task}\n')
                    f.write(f'{subject},{group},{rel_betaR},right,contra,{visit},{task}\n')
                elif subject in RIGHT:
                    f.write(f'{subject},{group},{rel_betaL},left,contra,{visit},{task}\n')
                    f.write(f'{subject},{group},{rel_betaR},right,ipsi,{visit},{task}\n')
                else:
                    f.write(f'{subject},{group},{rel_betaL},left,,{visit},{task}\n')
                    f.write(f'{subject},{group},{rel_betaR},right,,{visit},{task}\n')

# make ERD ERS

In [ ]:
import pandas as pd

for subject in subjects:
    for visit in ['visit1', 'visit2']:
        trigger_file = f'{subject[:5]}_{visit}_visual_triggers.csv' # subject[:5] to combine R26672 and R2667
        if not os.path.exists(f'{meg_folder}/{subject[:5]}/{trigger_file}'):
            print(f'file not found: {trigger_file}')
            continue
        print(trigger_file)
        triggers = pd.read_csv(f'{meg_folder}/{subject[:5]}/{trigger_file}')
        for task in ['mm', 'pn', 'pd']:
            rawfile = f'{subject}_{visit}_visual_{task}-raw.fif'
            if not os.path.exists(f'{meg_folder}/{subject}/{rawfile}'):
                print(f'file not found: {rawfile}')
                continue
            print(f'loading: {rawfile}')
            raw = mne.io.read_raw_fif(f'{meg_folder}/{subject}/{rawfile}')

            # source localization
            covfile = f'{meg_folder}/{subject}/{rawfile[:-7]}cov.fif'
            fwdfile = f'{meg_folder}/{subject}/{subject}_{visit}_visual-fwd.fif'
            fwd = mne.read_forward_solution(fwdfile)
            cov = mne.read_cov(covfile)
            invsol = mne.minimum_norm.make_inverse_operator(raw.info, fwd, cov, fixed=True, depth=0.8)
            stc1 = mne.minimum_norm.apply_inverse_raw(raw, invsol, lambda2 = 1, method='MNE')
            snd1 = load.fiff.stc_ndvar(stc1, 'fsaverage', 'ico-4', subjects_dir=subjects_dir)
            rolandicROI = list(set([l for l in snd1.source.parc.as_labels() if 'central' in l]))
            snd1 = snd1.sub(source=rolandicROI)
            
            tstart = load.unpickle(f'{meg_folder}/{subject}/{rawfile[:-8]}_tstart.pkl')
            snd1 = NDVar(snd1.x, (snd1.source, UTS(tstart, snd1.time.tstep, snd1.x.shape[1])))
            
            epochsR = []
            epochsL = []
            specgramsL = []
            specgramsR = []
            specgramsL_lh = []
            specgramsL_rh = []
            specgramsR_lh = []
            specgramsR_rh = []
            task_trigs = triggers[triggers['task']==task]
            i = 1
            for tt, side in zip(task_trigs['t_start'], task_trigs['button_side']):
                t = tt
                if subject == 'R26672':
                    t -=  1214 # get correct trigger times
                print(i, len(task_trigs['t_start']), t, ' '*20, end='\r')
                i += 1
                if t+3 > snd1.time.tmax:
                    print(f'{t+3} > {snd1.time.tmax}')
                    continue
                epoch = snd1.sub(time=(t-3,t+3))
                
                # make spectrogram using morlet wavelets
                freqs = np.logspace(*np.log10([6, 35]), num=20)
                n_cycles = freqs / 2.
                fs = 1/epoch.time.tstep
                specgram = mne.time_frequency.tfr_array_morlet(epoch.x[np.newaxis,:,:], fs, freqs=freqs, n_cycles=n_cycles, use_fft=True,
                                n_jobs=1, decim=1)
                specgram = NDVar(np.abs(specgram[0])**2, (epoch.source, Scalar('frequency', freqs), UTS(-3, 1/fs, specgram[0].shape[2])))
                specgram_lh = specgram.sub(source='lh').mean('source')
                specgram_rh = specgram.sub(source='rh').mean('source')
                specgram = specgram.mean('source')
                
                if side == 'left':
                    epochsL.append(epoch)
                    specgramsL.append(specgram)
                    specgramsL_lh.append(specgram_lh)
                    specgramsL_rh.append(specgram_rh)
                elif side == 'right':
                    epochsR.append(epoch)
                    specgramsR.append(specgram)
                    specgramsR_lh.append(specgram_lh)
                    specgramsR_rh.append(specgram_rh)
                
                
            save.pickle(epochsL, f'{output_folder}/{subject[:5]}_{visit}_visual_{task}_buttonL_epochs.pkl')
            save.pickle(epochsR, f'{output_folder}/{subject[:5]}_{visit}_visual_{task}_buttonR_epochs.pkl')
            save.pickle(specgramsL, f'{output_folder}/{subject[:5]}_{visit}_visual_{task}_button_specgramsL.pkl')
            save.pickle(specgramsR, f'{output_folder}/{subject[:5]}_{visit}_visual_{task}_button_specgramsR.pkl')
            save.pickle(specgramsL_lh, f'{output_folder}/{subject[:5]}_{visit}_visual_{task}_button_specgramsL_lh.pkl')
            save.pickle(specgramsR_lh, f'{output_folder}/{subject[:5]}_{visit}_visual_{task}_button_specgramsR_lh.pkl')
            save.pickle(specgramsL_rh, f'{output_folder}/{subject[:5]}_{visit}_visual_{task}_button_specgramsL_rh.pkl')
            save.pickle(specgramsR_rh, f'{output_folder}/{subject[:5]}_{visit}_visual_{task}_button_specgramsR_rh.pkl')

# write CSV file ERD ERS

In [ ]:
# functions for computing ERD, ERS
def get_ERD_ERS(ev, f1=13, f2=25):
    tbaseline = (-2.9,-2)
    RELBASE = ev.sub(frequency=(f1,f2)).mean('frequency').sub(time=tbaseline).mean('time') / ev.sub(time=tbaseline).mean('frequency').mean('time')
    ev = ev.sub(frequency=(f1,f2)).mean('frequency')
    BASE = ev.sub(time=tbaseline).mean('time')
    normf = ev.sub(time=tbaseline).mean('time')
    ts1 = -1
    te1 = 0.5
    ERD = -(ev.sub(time=(ts1,te1)).mean('time') - normf)/normf
    ts2 = 0.5
    te2 = 2.5
    ERS = (ev.sub(time=(ts2,te2)).mean('time') - normf)/normf
    if ERD <= 0:
        ERD = ''
    if ERS <= 0:
        ERS = ''
    return ERD, ERS, BASE, RELBASE

def get_ERD_ERS_trials(ev):
    ERDs = []
    ERSs = []
    BASEs = []
    RELBASEs = []
    ntrials = 0  
    for ii in range(len(ev)):
        ERD, ERS, BASE, RELBASE = get_ERD_ERS(ev[ii])
        if ERD!='':
            ERDs.append(ERD)
        if ERS!='':
            ERSs.append(ERS)        
        BASEs.append(BASE)
        RELBASEs.append(RELBASE)
        ntrials += 1
    if len(ERDs) == 0:
        ERD = ''
    else:
        ERD = np.sum(ERDs)/ntrials
    if len(ERSs) == 0:
        ERS = ''
    else:
        ERS = np.sum(ERSs)/ntrials
    BASE = np.sum(BASEs)/ntrials
    RELBASE = np.sum(RELBASEs)/ntrials
    return ERD, ERS, BASE, RELBASE



CONTROLS = ['R2517', 'R2519', 'R2520', 'R2521', 'R2525', 'R2528', 'R2496', 'R2673',] 
PATIENTS = ['R2527', 'R2540', 'R2546', 'R2598', 'R2615', 'R2617', 'R2664', 'R2667', 'R2668',]
subjects = CONTROLS + PATIENTS
# lesion hemisphere
LEFT =  ['R2527', 'R2540', 'R2667', 'R2668']
RIGHT = ['R2546', 'R2598', 'R2615', 'R2617', 'R2664']

beta_band = (13, 25)
psdrange = (2, 40)

outfile = 'ERD_ERS.csv'
with open(outfile, 'w+') as f:
    f.write(f'subject,group,value,metric,visit,task\n') # column headings
    
# lesion hemisphere button
outfile2 = 'ERD_ERS_lesionhemi.csv'
with open(outfile2, 'w+') as f:
    f.write(f'subject,group,value,metric,hemi,lesion,visit,task\n') # column headings 
    
for subject in subjects:
    if subject == 'R26672':
        continue
    for visit in ['visit1', 'visit2']:
        for task in ['mm', 'pn', 'pd']:
            infile = f'{subject}_{visit}_visual_{task}_button_specgramsL.pkl'
            if not os.path.exists(f'{output_folder}/{infile}'):
                print(f'file not found: {infile}')
                continue
            print(f'loading: {infile}')
            specL = load.unpickle(f'{output_folder}/{infile}')
            specR = load.unpickle(f'{output_folder}/{subject}_{visit}_visual_{task}_button_specgramsR.pkl')
            
            spec = combine([combine(specL),combine(specR)])
            ERD, ERS, BASE, RELBASE = get_ERD_ERS_trials(spec)
            
            if subject in CONTROLS:
                group = 'C'
            else:
                group = 'P'
            
            spec_lh = combine([combine(specR_lh), combine(specL_lh)])
            spec_rh = combine([combine(specR_rh), combine(specL_rh)])
            
            ERD_lh, ERS_lh, _, _ = get_ERD_ERS_trials(spec_lh)
            ERD_rh, ERS_rh, _, _ = get_ERD_ERS_trials(spec_rh)
            
            
            with open(outfile, 'a+') as f:
                f.write(f'{subject},{group},{ERD},ERD,{visit},{task}\n')
                f.write(f'{subject},{group},{ERS},ERS,{visit},{task}\n')
                f.write(f'{subject},{group},{BASE},BASE,{visit},{task}\n')
                f.write(f'{subject},{group},{RELBASE},RELBASE,{visit},{task}\n')

            with open(outfile2, 'a+') as f:
                if subject in LEFT:
                    f.write(f'{subject},{group},{ERD_lh},ERD,left,ipsi,{visit},{task}\n')
                    f.write(f'{subject},{group},{ERS_lh},ERS,left,ipsi,{visit},{task}\n')
                    f.write(f'{subject},{group},{ERD_rh},ERD,right,contra,{visit},{task}\n')
                    f.write(f'{subject},{group},{ERS_rh},ERS,right,contra,{visit},{task}\n')
                elif subject in RIGHT:
                    f.write(f'{subject},{group},{ERD_lh},ERD,left,contra,{visit},{task}\n')
                    f.write(f'{subject},{group},{ERS_lh},ERS,left,contra,{visit},{task}\n')
                    f.write(f'{subject},{group},{ERD_rh},ERD,right,ipsi,{visit},{task}\n')
                    f.write(f'{subject},{group},{ERS_rh},ERS,right,ipsi,{visit},{task}\n')
                else:
                    f.write(f'{subject},{group},{ERD_lh},ERD,left,,{visit},{task}\n')
                    f.write(f'{subject},{group},{ERS_lh},ERS,left,,{visit},{task}\n')
                    f.write(f'{subject},{group},{ERD_rh},ERD,right,,{visit},{task}\n')
                    f.write(f'{subject},{group},{ERS_rh},ERS,right,,{visit},{task}\n')